# Make QSUBs for Data Quality Downsampling

In [1]:
import numpy as np
from pathlib import Path
import os, re, sys, subprocess, tempfile
import numpy as np


def StartSHsub(filename):
    #server w/out scheduler

    f = open(filename, "w")
    f.write("#!/bin/bash\n\n")
    f.write("set -e\n")
    f.write("\n")
    f.write("conda activate downsampling\n")
    f.write("start=$(date +%s)\n")
    return f

def StartQsub(filename, nproc, memperproc, wd):
    #server w/ SGE scheduler
    
    f = open(filename, "w")
    f.write("#!/bin/bash\n\n")
    f.write("#$ -S /bin/bash\n")
    #f.write("#$ -cwd\n") # job should run in the current working directory
    f.write(f"#$ -wd {wd}\n") 
    f.write("#$ -j y\n") # STDERR and STDOUT should be joined
    f.write(f"#$ -l mem_free={memperproc}G\n") 
    f.write("#$ -l scratch=25G\n") # job requires up to 2 GiB of local /scratch space
    f.write(f"#$ -pe smp {nproc}\n") #the job will be allotted n slots (“cores”) on a single machine
    f.write("#$ -l h_rt=24:00:00\n")
    f.write("set -e\n")
    f.write("\n")
    f.write("module load CBI miniforge3\n")
    f.write("conda activate downsampling\n")
    
    f.write("\n")
    f.write("start=$(date +%s)\n")
    return f

def CloseSub(f, server):
    if (server == "wynton"): f.write('[[ -n "$JOB_ID" ]] && qstat -j "$JOB_ID"\n')
    f.write('echo "Elapsed Time: $(($end-$start)) seconds"\n')
    
    f.close()
    return

In [ ]:
server="wynton"
maxvmem = 300 #GB
nproc = 8
memperproc = round(maxvmem/nproc)
nrep = 3

In [ ]:
out_dir = "04_downsampling/02_cells"
subtype = "cells"
vals = ["1000", "2500", "5000", "7500", "10000", "20000", "30000", "40000", "50000"]

In [ ]:
out_dir = "04_downsampling/03_reads"
subtype = "reads"
vals = ["1e6", "5e6", "1e7", "5e7", "1e8", "5e8", "1e9"]

In [ ]:
out_dir = "04_downsampling/04_frip/"
subtype = "frip"
vals = ["0.05", "0.1", "0.2", "0.25", "0.35", "0.4", "0.6", "0.8"]

In [ ]:
qsub_dir = f"{out_dir}/qsubs/"
Path(f"{qsub_dir}").mkdir(parents=True, exist_ok=True)

if subtype == "reads" : short = "r"
elif subtype == "cells" : short = "c" 
elif subtype == "frip" : short = "f"
else: print("SUBTYPE NOT RECOGNIZED")
sge_ncpu = "${NSLOTS:-1}"

for val in vals:
    if subtype == "frip": group = f"{short}{val.replace('.', '')}"
    else: group = f"{short}{val}"
    print(group)

    Path(f"{out_dir}/{group}/").mkdir(parents=True, exist_ok=True)
    for celltype in ["HEPG2", "GM12878", "K562", "MCF7", "SKNSH"]: 
            
        qsub_script = f"{qsub_dir}/{celltype}_{group}.sh"
        if (server == "wynton"): 
            f = StartQsub(qsub_script, nproc, memperproc, wd = qsub_dir)
        if (server == "bueno"): 
            f = StartSHsub(qsub_script)
            sge_ncpu = nproc
            
        for seed in np.random.choice(range(1,100), nrep, replace=False):
            
            f.write(f"echo 'working on cell type: {celltype}_{group}_s{seed}'\n")
            
            command = f'''scBAMpler sampler \\
            -i /04_downsampling/00_celldicts/{celltype}.pickle \\
            -o {out_dir}/{group}/{celltype}_{group}_s{seed} \\
            --downsample_by {subtype} \\
            --downsample_to {val} \\
            --seed {seed} \\
            --nproc {sge_ncpu}  \\
            -b ../03_filtered_bams/{celltype}-cellFilt.bam\n\n'''
            f.write(command)
            
        CloseSub(f, server)

# Once finished, gather all summary statistics together

In [ ]:
######### BUILD SUMMARY STATS ###########

import os, re
import pandas as pd
import numpy as np

def _remove_strings(text):
    #remove common strings from test. 
    patterns = [r'\.summary\.txt', r'\.mat', r'-cellFilt']
    for pattern in patterns:
        text = re.sub(pattern, '', text)
    return text


def find_mat_files(directory):
    mat_files = []
    exclude_dirs = ["old", "qsubs", "older"]  # Directories to exclude

    for root, dirs, files in os.walk(directory):
        # Modify the dirs list in place to exclude specific directories
        dirs[:] = [d for d in dirs if d not in exclude_dirs]

        for file in files:
            if file.endswith(".summary.txt"):
                subdirectory = os.path.basename(root)
                parent_directory = os.path.basename(os.path.dirname(root))
                mat_files.append((parent_directory, subdirectory, file))
    
    return mat_files


data_dir = "/pollard/data/projects/aseveritt/encode_snatacseq/04_downsampling/"
all_files = find_mat_files(data_dir)
sum_stats = pd.DataFrame()

for d in all_files:
    pdd = d[0]; sd=d[1]; f=d[2]
    if pdd == "04_downsampling": pdd=""
    tmp = pd.read_csv(f"{data_dir}/{pdd}/{sd}/{f}", sep="\t", header=None, comment='#', index_col=0)
    tmp.columns = [_remove_strings(f)]
    sum_stats = sum_stats.merge(tmp, left_index=True, right_index=True, how="outer")

sum_stats = sum_stats.drop(['bam', 'input_file', "peak"])
fix = sum_stats.columns[sum_stats.loc[["sampling_type"]].isna().any()].tolist()
sum_stats.loc["sampling_type", fix] = "baseline"
sum_stats.loc["cell_line"] = [f.split('_')[0] for f in sum_stats.columns]
sum_stats.T.to_csv("/pollard/data/projects/aseveritt/encode_snatacseq/04_downsampling/sampling_stats.csv")
